# 2DOF Example Region Growth Visualization

In this notebook, we construct a 2DOF plant where we can visualize both the task and configuration space. Initial regions are generated using a heuristic which are then certified. We demonstrate various methods for proposing and growing regions using the bilinear alternation proposed in "Certified Polyhedral Decompositions of Collision-Free Configuration Space"

**Please copy a Mosek License to mosek.lic to run this** Failure to do so will result in the Scs solver being called which is too inaccurate for many of these problem.

Notebook runtime: less than 15 minutes

In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import os
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver

In [ ]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint. Click on the two links at the bottom to view the plant and the configuration space.



In [ ]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_file = "assets/oneDOF_iiwa7_with_box_collision.sdf"
with open(oneDOF_iiwa_file, 'r') as f:
    oneDOF_iiwa_string = f.read()
box_asset_file = "assets/box_small.urdf"
with open(box_asset_file, 'r') as f:
    box_asset_string = f.read()

models = []
models.append(parser.AddModelFromFile(box_asset_file))
models.append(parser.AddModelFromFile(oneDOF_iiwa_file, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_file,  'left_sweeper'))

locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_polytope, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [ ]:
# Some seedpoints
seed_points_q = np.array([   
                              [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.5,-1.08],
                              [-1.06, 0.48]
                              ])

seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
start = seed_points[0]
end = seed_points[-1]

visualizer.plot_cspace_points(seed_points, "/iris_seed_points", radius = 0.02)
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region.
default_alpha = 0.2
colors_dict = {
    0: Rgba(0.565, 0.565, 0.565, default_alpha), # gray
    1: Rgba(0.118, 0.533, 0.898, default_alpha), # bluish
    2: Rgba(1,     0.757, 0.027, default_alpha), # gold
    3: Rgba(0,     0.549, 0.024, default_alpha), # green   
    4: Rgba(0.055, 0.914, 0.929, default_alpha), # teal 
}

initial_regions = [(make_default_polytope_at_point(s), colors_dict[i]) for i, s in enumerate(seed_points)]
visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
                            wireframe = False,
                           opacity = default_alpha)
visualizer.show_res_q(q_star)

## First we set up some options for our different certification modes

It is **very** important that you upload a MOSEK license into the file `mosek.lic` in order to run the rest of the notebook. If you do not, the solver `SCS` will be used which is much slower and likely will be too inaccurate to solve many of the problems.

In [ ]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)



solver_id = MosekSolver.id() if MosekSolver().available() else ScsSolver.id()

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = False
find_separation_certificate_given_polytope_options.solver_id = solver_id

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-8
find_polytope_given_lagrangian_option.solver_id = solver_id

bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 5 # Setting this to a high number will lead to more fill
bilinear_alternation_options.convergence_tol = 1e-3
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 100

# Growing regions with bilinear alternations search

### As the initial regions are fairly small, they won't contain collisions and so can be directly fed into the bilinear alternation algorithm.

In [ ]:
# We grow certified regions around each seedpoint using bilinear alternation.
from time import perf_counter
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
times_per_region = []
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    t_start = time.perf_counter()
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    t_end = time.perf_counter()
    times_per_region.append(t_end - t_start)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(cert.certified_polytope,
                                                             cert, color) for cert in result]



In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(bilinear_alternation_results_by_seed_point.values()):
    group_name = f"/bil_alt/seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

# Growing regions with binary search

### While the bilinear alternation scheme has the flexibility to search for fairly flexible polytopes, it can be relatively slow. We can search for larger regions faster by uniformly growing our polytopes using binary search

In [ ]:
# We grow certified regions around each seedpoint using binary search.
binary_search_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cert = cspace_free_polytope.BinarySearch(set(),initial_region.A(),
                                               initial_region.b(),s,binary_search_options)
    binary_search_results_by_seed_point[tuple(s)] = [(cert.certified_polytope, cert, color)]
                                                     


In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_search_results_by_seed_point.values()):
    group_name = f"/bin_search/seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

## Combining Binary Search and Bilinear Alternation
### Of course, we can get the best of both worlds by combining the two methods

In [ ]:
# we use this to back off a bit from the binary search results for numerical reasons
def scale_polytope_about_point(hpoly, s, scale):
    if not hpoly.PointInSet(s):
        raise ValueError(f"The point s must be in the HPolyhedron")
    b_scaled = scale * hpoly.b() + (1 - scale) * hpoly.A() @ s
    return HPolyhedron(hpoly.A(), b_scaled)
    

In [ ]:
# Now we grow certified regions around each seedpoint using bilinear alternation 
# starting from the binary certified regions.

binary_and_bilinear_certified_regions = binary_search_results_by_seed_point.copy()
for i, (s, val) in enumerate(binary_and_bilinear_certified_regions.items()):
    (initial_region, cert, color) = val[0]
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cur_s = np.array(s)
    bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
    # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
    initial_region = scale_polytope_about_point(initial_region, cur_s, 0.90)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      initial_region.A(),
                                                                      initial_region.b(), 
                                                                      bilinear_alternation_options)
    binary_and_bilinear_certified_regions[s] += [(result.certified_polytope, result, color) for result in certificates]

In [ ]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_and_bilinear_certified_regions.values()):
    group_name = f"/bin_then_bil/seed_point_{i}/"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2,
                                                               fill = False)
    visualizer.add_group_of_regions_and_certs_to_visualization([result[-1]], group_name+"/final" , 
                                                               wireframe = False, opacity = 0.2,
                                                               line_width = 1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7e82e4f5-f47a-475a-aad3-c88093ed36c6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>